In [1]:
#|export

# Use the FastAi tabular library for the Titanic dataset
from fastai.tabular.all import *
import pandas as pd

In [15]:
#|export
df = pd.read_csv("../datasets/train.csv")
print(df.columns)
print(df.size)
df.head()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
10692


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
categorical_features = ["Pclass", "Sex", "Embarked"]
continuous_features = ["Age", "SibSp", "Parch", "Fare"]
y_column = "Survived"

In [22]:
data_split = RandomSplitter(valid_pct=0.2, seed=42)(range_of(df))

In [18]:
doc(range_of)

In [31]:
tabular_obj = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names=categorical_features,
                   cont_names=continuous_features,
                   y_names=y_column,
                   y_block=CategoryBlock,
                   splits=data_split)

In [24]:
doc(TabularPandas)

In [32]:
tabular_obj.xs.iloc[:5]

,Pclass,Sex,Embarked,Age_na,Age,SibSp,Parch,Fare
788,3,2,3,1,-2.182087,0.461591,2.034995,-0.226999
525,3,2,2,1,0.861920,-0.462887,-0.464491,-0.477002
821,3,2,3,1,-0.178437,-0.462887,-0.464491,-0.459214
253,3,2,3,1,0.052753,0.461591,-0.464491,-0.314232
374,3,1,3,1,-2.027960,2.310547,0.785252,-0.217253


In [33]:
dls = tabular_obj.dataloaders(bs=64)
dls.show_batch()

,Pclass,Sex,Embarked,Age_na,Age,SibSp,Parch,Fare,Survived
0,2,male,S,False,21.000000,1.000000e+00,-9.897945e-09,1.150000e+01,0
1,2,male,S,False,2.999998,1.000000e+00,1.000000e+00,2.600000e+01,1
2,2,male,C,False,30.000000,1.000000e+00,-9.897945e-09,2.400000e+01,0
3,3,male,S,False,21.000000,1.689237e-09,-9.897945e-09,1.610000e+01,0
4,1,male,S,False,38.000000,1.689237e-09,-9.897945e-09,7.312175e-07,0
5,2,female,S,False,19.000000,1.000000e+00,-9.897945e-09,2.600000e+01,1
6,1,female,#na#,False,38.000000,1.689237e-09,-9.897945e-09,8.000000e+01,1
7,3,female,S,False,41.000000,1.689237e-09,2.000000e+00,2.021250e+01,0
8,3,female,Q,False,32.000000,1.000000e+00,1.000000e+00,1.550000e+01,0
9,3,female,S,False,18.000000,1.000000e+00,-9.897945e-09,1.780000e+01,0


In [35]:
learn = tabular_learner(dls, metrics="accuracy")
learn.fit_one_cycle(1)

RuntimeError: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	The MPS backend is supported on MacOS 12.3+.Current OS version can be queried using `sw_vers`